CS524: Introduction to Optimization Lecture 16
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 11, 2023
--------------

## GDX

- Stand for GAMS Data Exchange
- GDX files are binary files that store data for use in gams models
- There exist several utilities to generate, include and look at these files, including API's in all popular languages
- Use the gdx viewer within Studio to see what is in a file or at command prompt use [GDXDUMP](https://www.gams.com/latest/docs/T_GDXDUMP.html)
- ["GAMS Data eXchange" GDX](https://www.gams.com/latest/docs/UG_GDX.html)
  
### Using GDX depends on phase in which it is employed
   1. if in Compilation phase: use $ control options
      - reading during compilation allows us to define the elements of a set, and use this set as a domain
    
   2. if in Execution phase: use command `execute_`

## Reading GDX Files into GAMS

The basic structure of reading the GDX file is as follows:

    $GDXIN filename
$LOAD foo
    $LOAD id=gdxid
$GDXIN

<i>Here</i>

- `$` means you are directed towards the compiler
- `filename` is the file that you want to read
- `foo` is a read-in data
    *(a GAMS symbol: could represent a set, parameter, variable or equation)*
- `id` is a GAMS symbol, 'gdxid' is a name in the gdx file


In [1]:
%load_ext gams.magic
m = gams.exchange_container

In [2]:
%%gams

$gdxin trnsport_output.gdx
Set m(*) markets
$load m=j
Parameter demand(m) demand at market m;
$load demand=b
$gdxin

<i>In example above:</i>

- `LOAD demand=b`: load in data as if it's static
- if you want to unload at compile time (typically you wouldn't), then you can use `$GDXOUT` and `$UNLOAD`


In [3]:
b=m.data['demand'].records
display(b);

,m,value
0,new-york,325.0
1,chicago,300.0
2,topeka,275.0


## If you are not using Studio: GDXDUMP
- GDXDUMP writes the contents of a GDX file as a GAMS formatted text file
- This will produce a listing of all the symbols in a gdx file
- Code below shows how to read data with GDXDUMP
- `$onEmpty` allows empty data statements for list or table formats. Note that by default, empty data statements will cause a compilation error.
- You can view the content using the command: `gdxdump trnsport.gdx`
- See use of this command in topbrass example, `topbrass6.gms`

## Execution Time Handling

- `execute_load` will read data from a GDX file during execution phase. <br />
- acts like an assignment statement, except that it does not merge the data read with the current data.
- It is a full replacement.<br />
For example: `execute_load ’filename’, id1, id2, ...`<br /><br />
- Following does a merge.<br />
For example: `execute_loadpoint ’filename’, id1, id2, ...`<br /><br />
- `execute_unload` will write data from a GDX file during execution phase. <br />
- replaces an existing file with that name. <br />
For example: `execute_unload ’filename’, id1, id2, ...`

<i>Example:</i><br />
This write the level and marginals of the x variables into a gdx file

    *=== Unload to GDX file
    execute_unload "tresults.gdx" x.L x.M

<i>Notes:</i>
- Can open the GDX file in Studio
- Adding <font color=blue>gdx=filename</font> unloads all symbols at the end of the run

In [4]:
%%gams
set J(*) 'barrier site'; 
alias (J,K);
set D(J,K) 'downstream barriers from j (not including j)', 
    ROOT(J) 'root nodes of forest';
parameter 
    pi(J) 'increase in probability of passage', 
    v(J) 'net habitat between J and its upstream neighbors', 
    c(J) 'cost of project at J', 
    b 'available budget', 
    pbar(J) 'current probability of passage at J';
$gdxin data-small.gdx
$load J,D,ROOT
$load pi,v,c,b,pbar
$gdxin

In [5]:
J=m.data['J'].records
display(J)

,uni,element_text
0,101518,
1,101519,
2,101520,
3,101521,
4,101522,
...,...,...
12536,155700,
12537,155701,
12538,155705,
12539,155706,


To do gams->gdx just use `execute_unload`

To do gdx->gams just use `$GDXIN` or `execute_load`

In [6]:
gams.reset()

### GDX reading and writing from python

First create the gdx file using gams with execute_unload

In [7]:
%%gams
$Title  A Transportation Problem (TRNSPORT,SEQ=1)
$Ontext

This problem finds a least cost shipping schedule that meets
requirements at markets and supplies at factories.


Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions. 
Princeton University Press, Princeton, New Jersey, 1963.

This formulation is described in detail in:
Rosenthal, R E, Chapter 2: A GAMS Tutorial. In GAMS: A User's Guide. 
The Scientific Press, Redwood City, California, 1988.

The line numbers will not match those in the book because of these 
comments.

$Offtext


  Sets
       i   canning plants   / seattle, san-diego /
       j   markets          / new-york, chicago, topeka / ;

  Parameters

       a(i)  capacity of plant i in cases
         /    seattle     350
              san-diego   600  /

       b(j)  demand at market j in cases
         /    new-york    325
              chicago     300
              topeka      275  / ;

  Table d(i,j)  distance in thousands of miles
                    new-york       chicago      topeka
      seattle          2.5           1.7          1.8
      san-diego        2.5           1.8          1.4  ;

  Scalar f  freight in dollars per case per thousand miles  /90/ ;

  Parameter c(i,j)  transport cost in thousands of dollars per case ;

            c(i,j) = f * d(i,j) / 1000 ;

  Variables
       x(i,j)  shipment quantities in cases
       z       total transportation costs in thousands of dollars ;

  Positive Variable x ;

  Equations
       cost        define objective function
       supply(i)   observe supply limit at plant i
       demand(j)   satisfy demand at market j ;

  cost ..        z  =e=  sum((i,j), c(i,j)*x(i,j)) ;

  supply(i) ..   sum(j, x(i,j))  =l=  a(i) ;

  demand(j) ..   sum(i, x(i,j))  =g=  b(j) ;

  Model transport /all/ ;

Solve transport using lp minimizing z ;

execute_unload "trnsport_output", i,j,a,b,c,d,f,x,z,cost,supply,demand;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),153.675,6,7,LP,CPLEX,0.002


# Read named gdx into python (use gams.transfer)
(could read/write to Jupyter using exchange_container as below)

In [8]:
import gams.transfer as gt

# create instance of gams gdx data
gdx = gt.Container() #system_directory='/Library/Frameworks/GAMS.framework/Resources'
# read in single items
gdx.read('trnsport_output.gdx',['i','j','x','z'])

x=gdx.data['x']
cost = gdx.data['z'].toValue()
# display variable x data as a dataframe
display(x.records,cost)
display(x.pivot(),x.pivot(value='marginal'))

,i,j,level,marginal,lower,upper,scale
0,seattle,new-york,50.0,0.000,0.0,inf,1.0
1,seattle,chicago,300.0,0.000,0.0,inf,1.0
2,seattle,topeka,0.0,0.036,0.0,inf,1.0
3,san-diego,new-york,275.0,0.000,0.0,inf,1.0
4,san-diego,chicago,0.0,0.009,0.0,inf,1.0
5,san-diego,topeka,275.0,0.000,0.0,inf,1.0


153.675

,new-york,chicago,topeka
seattle,50.0,300.0,0.0
san-diego,275.0,0.0,275.0


,new-york,chicago,topeka
seattle,0.0,0.000,0.036
san-diego,0.0,0.009,0.000


# Write from python (tuples) into gdx (use gams.transfer)

In [9]:
gdx = gt.Container()

i = gdx.addSet('i', ['*'], records=['seattle', 'san-diego'], description='canning plants')
j = gdx.addSet('j', ['*'], description='markets', records=['new-york', 'chicago', 'topeka'])
c = gdx.addParameter('c', domain=['i', 'j'],
  records = [('seattle', 'new-york', 0.225),
   ('seattle', 'chicago', 0.153),
   ('seattle', 'topeka', 0.162),
   ('san-diego', 'new-york', 0.225),
   ('san-diego', 'chicago', 0.162),
   ('san-diego', 'topeka', 0.126)],
   description='transport cost in thousands of dollars per case');

gdx.write('trnsport_output_alt.gdx', compress=False)

# Write from python (dataframe) into gdx (use gams.transfer)

In [10]:
import pandas as pd

dist = pd.DataFrame(
    [
        ("seattle", "new-york", 2.5),
        ("seattle", "chicago", 1.7),
        ("seattle", "topeka", 1.8),
        ("san-diego", "new-york", 2.5),
        ("san-diego", "chicago", 1.8),
        ("san-diego", "topeka", 1.4),
    ],
    columns=["from", "to", "thousand_miles"],
)
d = gdx.addParameter('d', domain=['i', 'j'], records = dist,
    description='transport in thousands of miles');

# c.setRecords(dist)

gdx.write('trnsport_output_df.gdx', compress=False)

# Read from GAMS via exchange container (without gdx file)

In [11]:
m = gams.exchange_container
x = m.data['x']
cost = m.data['z'].toValue()
display(x.records,cost)
display(x.pivot(),x.pivot(value='marginal'))

,i,j,level,marginal,lower,upper,scale
0,seattle,new-york,50.0,0.000,0.0,inf,1.0
1,seattle,chicago,300.0,0.000,0.0,inf,1.0
2,seattle,topeka,0.0,0.036,0.0,inf,1.0
3,san-diego,new-york,275.0,0.000,0.0,inf,1.0
4,san-diego,chicago,0.0,0.009,0.0,inf,1.0
5,san-diego,topeka,275.0,0.000,0.0,inf,1.0


153.675

,new-york,chicago,topeka
seattle,50.0,300.0,0.0
san-diego,275.0,0.0,275.0


,new-york,chicago,topeka
seattle,0.0,0.000,0.036
san-diego,0.0,0.009,0.000


In [12]:
gams.gams_cleanup('--closedown')